# Éves kiértékelés és tisztítás

In [15]:
import pandas as pd
import numpy as np
from datetime import timedelta
from numpy.core.numeric import Inf, NaN
from datetime import date
local_path = '/Users/csizi/Development/marathon_statistics/dataset/csv/'
df = pd.read_csv(local_path + 'BSZM_2020.csv', encoding = "ISO-8859-1", 
                    sep=';', header=0)
df['sz.év.'] = df['sz.év.'].replace(NaN, 9999)
df['sz.év.'] = df['sz.év.'].astype(int)
df['curryear'] = date.today().year
df['kor'] = df['curryear'] - df['sz.év.']
df.drop(['rajtszám', 'kat.h.', 'curryear'], axis = 1, inplace=True)    
df.drop(['1.nap/1.idõ', '1.nap/2.idõ', '1.nap/3.idõ', '2.nap/1.idõ', '2.nap/2.idõ', '2.nap/3.idõ', '3.nap /1.idõ', '3.nap/2.idõ', '3.nap/3.idõ', '4.nap/1.idõ', '4.nap/2.idõ', '4.nap/3.idõ'], axis = 1, inplace=True)
df["befejezte"] = np.where(df['Megtett táv (km)'] < df['Megtett táv (km)'].max(), False, True)
df = df.replace('99:59:59', np.nan)
df['eredmény'] = pd.to_timedelta(df['eredmény'])
df['1.nap összidõ'] = pd.to_timedelta(df['1.nap összidõ'])
df['2.nap összidõ'] = pd.to_timedelta(df['2.nap összidõ'])
df['3.nap összidõ'] = pd.to_timedelta(df['3.nap összidõ'])
df['4.nap összidõ'] = pd.to_timedelta(df['4.nap összidõ'])
eredmény = ['1.nap összidõ', '2.nap összidõ', '3.nap összidõ', '4.nap összidõ']
df['eredmény'] = df[eredmény].sum(axis=1, skipna=True)
df['eredmény'] = df['eredmény'] - pd.to_timedelta(df['eredmény'].dt.days, unit='h')
df["eredmény(perc)"] = df['eredmény'].astype('timedelta64[m]')
df["átlagos tempó (perc/km)"] = df['eredmény'].astype('timedelta64[m]')/df['Megtett táv (km)']
df['sebesség kategória'] = pd.qcut(df['átlagos tempó (perc/km)'], 5, labels=['gyors', 'közepesen gyors', 'közepes', 'közepesen lassú', 'lassú'])
df.loc[df['eredmény(perc)'] == 0.0] = np.nan
df.drop_duplicates(inplace=True)
df.loc[df['átlagos tempó (perc/km)'] < 3] = np.nan
df.loc[df['átlagos tempó (perc/km)'] == np.inf] = np.nan
df.dropna(axis=0, how='all', inplace=True)

,Hely.,Név/Csapatnév,sz.év.,Kategória,Ország,Csapat,Város,nem,1.nap összidõ,2.nap összidõ,3.nap összidõ,4.nap összidõ,eredmény,Megtett táv (km),kor,befejezte,eredmény(perc),átlagos tempó (perc/km),sebesség kategória
0,1.0,Beda Szabolcs,1977.0,F3,HUN,ELTE SE Sashegyi gepárdok,Budapest,F,0 days 03:08:51,0 days 03:30:32,0 days 02:53:54,0 days 03:28:51,0 days 13:02:08,196.0,44.0,1.0,782.0,3.989796,gyors
1,2.0,Bódis Tamás,1988.0,F1,HUN,ELTE SE Sashegyi gepárdok,Budapest,F,0 days 03:13:57,0 days 03:40:22,0 days 02:48:22,0 days 03:24:07,0 days 13:06:48,196.0,33.0,1.0,786.0,4.010204,gyors
2,3.0,"Theisinger, Wolfgang",1982.0,F2,GER,RC Abschnitt Mitte,Regenstauf,F,0 days 03:23:21,0 days 03:53:42,0 days 03:04:38,0 days 03:43:54,0 days 14:05:35,196.0,39.0,1.0,845.0,4.311224,gyors
3,4.0,Strifler Attila,1986.0,F1,HUN,NaN,Gyula,F,0 days 03:31:41,0 days 03:59:24,0 days 03:05:44,0 days 03:45:58,0 days 14:22:47,196.0,35.0,1.0,862.0,4.397959,gyors
4,5.0,Szedlák Csaba,1978.0,F3,HUN,Vitalitás SE,Budapest,F,0 days 03:31:31,0 days 04:00:30,0 days 03:17:03,0 days 03:53:35,0 days 14:42:39,196.0,43.0,1.0,882.0,4.500000,gyors
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,NaN,Czeglédy Csaba dr.,1976.0,F3,HUN,NaN,Szombathely,F,0 days 05:35:16,NaT,NaT,NaT,0 days 05:35:16,48.0,45.0,0.0,335.0,6.979167,lassú
290,NaN,Bencsik Béla,1968.0,F4,HUN,NaN,Gyõrújbarát,F,NaT,NaT,NaT,0 days 07:53:21,0 days 07:53:21,82.0,53.0,0.0,473.0,5.768293,közepesen gyors
291,NaN,Ozsvár Tihamér,1969.0,F4,HUN,NaN,Csantavér,F,0 days 04:34:48,NaT,0 days 04:12:19,NaT,0 days 08:47:07,108.0,52.0,0.0,527.0,4.879630,gyors
293,NaN,Nagy Attila,1978.0,F3,HUN,NaN,Budapest,F,0 days 05:08:52,NaT,NaT,NaT,0 days 05:08:52,48.0,43.0,0.0,308.0,6.416667,közepes


In [17]:
df.loc[df['Hely.'] == np.nan]

,Hely.,Név/Csapatnév,sz.év.,Kategória,Ország,Csapat,Város,nem,1.nap összidõ,2.nap összidõ,3.nap összidõ,4.nap összidõ,eredmény,Megtett táv (km),kor,befejezte,eredmény(perc),átlagos tempó (perc/km),sebesség kategória
